In [1]:
from datasets.air import Air

import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

from torch_geometric.utils import negative_sampling
from torch_geometric.nn import GCNConv

/home/honglu/Softwares/anaconda3/envs/dhypr_lib/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

In [3]:
dataset = Air('link_prediction')
fold_0 = dataset.get(0)

Extracting air traffic control dataset from /home/honglu/advith/dhypr-lib/dhypr/datasets/air/link_prediction/raw/download.tsv.maayan-faa.tar.bz2
Processing fold 0


Processing...


k=1 diffusion_in   took 6.67572021484375e-06 s
k=1 diffusion_out   took 2.1457672119140625e-06 s
k=1 neighbor_in   took 0.036828041076660156 s
k=1 neighbor_out   took 0.029230356216430664 s
Processing fold 1
k=1 diffusion_in   took 4.5299530029296875e-06 s
k=1 diffusion_out   took 2.1457672119140625e-06 s
k=1 neighbor_in   took 0.03281760215759277 s
k=1 neighbor_out   took 0.02689671516418457 s
Processing fold 2
k=1 diffusion_in   took 5.9604644775390625e-06 s
k=1 diffusion_out   took 2.384185791015625e-06 s
k=1 neighbor_in   took 0.044298648834228516 s
k=1 neighbor_out   took 0.04185295104980469 s
Processing fold 3
k=1 diffusion_in   took 5.0067901611328125e-06 s
k=1 diffusion_out   took 2.6226043701171875e-06 s
k=1 neighbor_in   took 0.03152275085449219 s
k=1 neighbor_out   took 0.027666568756103516 s
Processing fold 4
k=1 diffusion_in   took 5.245208740234375e-06 s
k=1 diffusion_out   took 2.384185791015625e-06 s
k=1 neighbor_in   took 0.04136824607849121 s
k=1 neighbor_out   took 0

Done!


In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(fold_0.num_nodes, 128)
        self.conv2 = GCNConv(128, 64)

    def encode(self):
        x = self.conv1(fold_0.x, fold_0.train_pos_edge_index) # convolution 1
        x = x.relu()
        return self.conv2(x, fold_0.train_pos_edge_index) # convolution 2

    def decode(self, z, pos_edge_index, neg_edge_index): # only pos and neg edges
        edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1) # concatenate pos and neg edges
        logits = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)  # dot product 
        return logits

    def decode_all(self, z): 
        prob_adj = z @ z.t() # get adj NxN
        return (prob_adj > 0).nonzero(as_tuple=False).t() # get predicted edge_list 

In [5]:
model, data = Net().to(device), fold_0.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

In [6]:

def get_link_labels(pos_edge_index, neg_edge_index):
    # returns a tensor:
    # [1,1,1,1,...,0,0,0,0,0,..] with the number of ones is equel to the lenght of pos_edge_index
    # and the number of zeros is equal to the length of neg_edge_index
    E = pos_edge_index.size(1) + neg_edge_index.size(1)
    link_labels = torch.zeros(E, dtype=torch.float, device=device)
    link_labels[:pos_edge_index.size(1)] = 1.
    return link_labels


def train():
    model.train()

    neg_edge_index = negative_sampling(
        edge_index=data.train_pos_edge_index, #positive edges
        num_nodes=data.num_nodes, # number of nodes
        num_neg_samples=data.train_pos_edge_index.size(1)) # number of neg_sample equal to number of pos_edges

    optimizer.zero_grad()
    
    z = model.encode() #encode
    link_logits = model.decode(z, data.train_pos_edge_index, neg_edge_index) # decode
    
    link_labels = get_link_labels(data.train_pos_edge_index, neg_edge_index)
    loss = F.binary_cross_entropy_with_logits(link_logits, link_labels)
    loss.backward()
    optimizer.step()

    return loss


@torch.no_grad()
def test():
    model.eval()
    perfs = []
    for prefix in ["val", "test"]:
        pos_edge_index = data[f'{prefix}_pos_edge_index']
        neg_edge_index = data[f'{prefix}_neg_edge_index']

        z = model.encode() # encode train
        link_logits = model.decode(z, pos_edge_index, neg_edge_index) # decode test or val
        link_probs = link_logits.sigmoid() # apply sigmoid
        
        link_labels = get_link_labels(pos_edge_index, neg_edge_index) # get link
        
        perfs.append(roc_auc_score(link_labels.cpu(), link_probs.cpu())) #compute roc_auc score
    return perfs

In [7]:
best_val_perf = test_perf = 0
for epoch in range(1, 101):
    train_loss = train()
    val_perf, tmp_test_perf = test()
    if val_perf > best_val_perf:
        best_val_perf = val_perf
        test_perf = tmp_test_perf
    log = 'Epoch: {:03d}, Loss: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    if epoch % 10 == 0:
        print(log.format(epoch, train_loss, best_val_perf, test_perf))

IndexError: index 1226 is out of bounds for dimension 0 with size 1226